# Create figures for GSWP3 atmospheric forcing
Plots to visualize some of the model inputs for atmospheric forcing from GSWP3, i.e., the dataset providing climatic observations driving model processes such as photosynthesis or evaporation.

After specifying the path to a case's `inputdata/` folder below, run all cells in this notebook. The most convenient way is clicking the "⏩" button contained in the navigation bar at the top. It will create image files called `atmosphere_[variable_name].png` in a folder with the given case name in this working directory - i.e., either a new folder `/notebooks/plot_input_data/[case_id]-[case_name]/` will appear or the files will be added to it if it already exists. Click the folder icon in the navigation bar to the left if JupyterLab currently does not display the File Browser. Feel free to adapt the code in this notebook according to your needs.

In [ ]:
# Add the full case name in quotation marks here! E.g.: "7733d6a59846cb18c051d598e968f695_alp1-test"
case_name = "579673982ba6f4c1cf6cb10d7ab413a6_bor1-2000-2001"

# Enter the case ID (aka. short site code) in quotation marks here! E.g.: "BOR1".
# ATTENTION! This is case sensitive, use the correct capitalization of letters!
case_id = "BOR1"

# Path to input data. Only change this line if you renamed the folder or moved it from the default location!
from pathlib import Path
inputdata_path = Path(f"../../data/{case_name}/inputdata/")

In the cell below, you can specify custom start and end dates for plotting a short time period with the original CLM5-GSWP3 temporal resolution, i.e. in 3 hourly timesteps. It shouldn't be longer than a year to maintain readability. Changing the time period can be useful if you have observational data that you roughly want to compare to the default inputs. The plots give you an idea which atmospheric data is needed for running the CLM in land-only mode - you could integrate your own observations for specialized experiments.

In [ ]:
# WARNING! Must be between 1901-01-01 and 2014-12-31!
subset_time_start = '2013-01-01'  # Format: YYYY-MM-DD
subset_time_end = '2013-12-31'  # Format: YYYY-MM-DD

<span style="color: red;">Warning!</span> The following code is written for the NorESM-LSP default atmospheric forcing, i.e. the GSWP3 dataset. If you provide different forcing data or changed the default save locations you need to adjust the scripts accordingly.

_No code cells below need to be altered unless you desire custom changes._

---

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import time

In [ ]:
# Root path to surface data
atm_forcing_data_path = Path(f"{inputdata_path}/atm/datm7/GSWP3v1/{case_id}")

In [ ]:
# open and combine all the monthly atmospheric NetCDF files present in the folder into an xarray dataset
start_time = time.time()
atm_forcing_data = xr.open_mfdataset(f'{atm_forcing_data_path}/*{case_id}*.nc', 
                                     concat_dim='time', 
                                     combine='nested', 
                                     decode_times=True)
print(f"--- Time it took to read the data: {round(float(time.time() - start_time), 3)} seconds ---")

In [ ]:
# Create case folder for output plots if necessary
out_dir_path = Path().absolute() / case_name

if not out_dir_path.is_dir():
    out_dir_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Set general plotting options
DPI = 150
TICK_LABEL_FONT_SIZE = 12
AX_LABEL_FONT_SIZE = 14
TITLE_FONT_SIZE = 16

## Full dataset period

In [ ]:
# Print dataset info
atm_forcing_data

#### Annual temperature, precipitation, and total incident solar radiation
The code cells below illustrate how the `xarray` and `matplotlib` libraries can be used for input data wrangling and plotting.

In [ ]:
# Temperature, resample to annual mean, convert from K to °C
temp_mean_year = atm_forcing_data["TBOT"].resample(time='1Y').mean() - 273.15

# Precipitation, convert from [mm H2O / sec] to total (3h=10800s model time step), resample to annual sum
precip_total_year = (atm_forcing_data["PRECTmms"] * 10800).resample(time='1Y').sum()

# Radiation, resample to annual mean
radiation_mean_year = atm_forcing_data["FSDS"].resample(time='1Y').mean()

In [ ]:
# Instantiate plot, one subplot for each variable
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1,
                                    figsize=(8, 8),
                                    dpi=DPI)

ax1.plot(range(len(temp_mean_year.time)),
         temp_mean_year.values.flatten(),
         color="#C0392B",
         linewidth=2,
         label="Annual mean temperature [°C]"
       )

ax2.bar(x=range(len(precip_total_year.time)),
        height=precip_total_year.values.flatten(),
        color="#0047AB",
        label="Total precipitation"
       )

ax3.plot(radiation_mean_year.time,
         radiation_mean_year.values.flatten(),
         color="#28282B",
         linewidth=2,
         label="Annual mean incident solar radiation [W/m²]"
        )

# Disable x axes for subplots 1 and 2
ax1.get_xaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)

# Other plot layout settings
ax1.set_title(f"{case_id}: GSWP3 forcing (1901-1914)", size=TITLE_FONT_SIZE)

ax1.set_ylabel("Annual mean 2m air\ntemperature [°C]", fontsize=AX_LABEL_FONT_SIZE)
ax2.set_ylabel("Annual total\nprecipitation [mm]", fontsize=AX_LABEL_FONT_SIZE)
ax3.set_ylabel("Annual mean incident\nsolar radiation [W/m²]", fontsize=AX_LABEL_FONT_SIZE)
ax3.set_xlabel("Year", fontsize=AX_LABEL_FONT_SIZE)

ax1.tick_params(axis='y', which='major', labelsize=TICK_LABEL_FONT_SIZE)
ax2.tick_params(axis='y', which='major', labelsize=TICK_LABEL_FONT_SIZE)
ax3.tick_params(axis='both', which='major', labelsize=TICK_LABEL_FONT_SIZE)

# Save figure
fig.tight_layout()
fig.savefig(fname=f"{out_dir_path}/atmosphere_yearly_temp_precip_rad.png", format='png')

---
## Short period

In [ ]:
atm_forcing_data

### Temperature, precipitation, total radiation, longwave radiation, humidity, wind, atm. pressure 

In [ ]:
variable_list = []
atm_forcing_subset = atm_forcing_data.sel(time=slice(subset_time_start, subset_time_end))

# Temperature, convert from K to °C
temp_subset = atm_forcing_subset["TBOT"] - 273.15
variable_list.append(temp_subset)

# Precipitation, convert from [mm H2O / sec] to total [mm] (data temp. resolution: 3h=10800s), calculate cumulative sum
precip_subset = (atm_forcing_subset["PRECTmms"] * 10800).cumsum()
variable_list.append(precip_subset)

# Total incident radiation
radation_total_subset = atm_forcing_subset["FSDS"]
variable_list.append(radation_total_subset)

# Incident longwave radiation
radation_longwave_subset = atm_forcing_subset["FLDS"]
variable_list.append(radation_longwave_subset)

# Specific humidity
humidity_subset = atm_forcing_subset["QBOT"]
variable_list.append(humidity_subset)

# Wind speed
wind_subset = atm_forcing_subset["WIND"]
variable_list.append(wind_subset)

# Atm. pressure, convert to hPa
pressure_subset = atm_forcing_subset["PSRF"] * 0.01
variable_list.append(pressure_subset)

In [ ]:
# Label dictionary
subset_plot_dict = {
    "TBOT": {
        "y_axis": "2m air temperature",
        "unit": "[°C]",
        "color": "#C0392B",
    },
    "PRECTmms": {
        "y_axis": "Cumulative precipitation",
        "unit": "[mm]",
        "color": "#0047AB",
    },
    "FSDS": {
        "y_axis": "Total incident solar radiation",
        "unit": "[W/m²]",
        "color": "#28282B",
    },
    "FLDS": {
        "y_axis": "Incident longwave radiation",
        "unit": "[W/m²]",
        "color": "#FE5654",
    },
    "QBOT": {
        "y_axis": "2m specific humidity",
        "unit": "[kg/kg]",
        "color": "#8EA9F6",
    },
    "WIND": {
        "y_axis": "2m wind speed",
        "unit": "[m/s]",
        "color": "#F00BF3",
    },
    "PSRF": {
        "y_axis": "Surface pressure",
        "unit": "[hPa]",
        "color": "#6B6B6B",
    },
}

In [ ]:
# Create plots for each variable
for variable in variable_list:
    
    # Get plot settings for current variable
    current_var_dict = subset_plot_dict.get(variable.name)
    
    # Instantiate plot
    fig, ax = plt.subplots(figsize=(6, 6), dpi=DPI)

    ax.plot(variable.time,
            variable.values.flatten(),
            color=current_var_dict.get('color'),
            linewidth=2,
           )
    
    # Other plot layout settings
    ax.set_title(f"{case_id}: {variable.name} - GSWP3 forcing\n{subset_time_start} -- {subset_time_end}", size=TITLE_FONT_SIZE)

    ax.set_ylabel(current_var_dict.get('y_axis') + " " + current_var_dict.get('unit'),
                  fontsize=AX_LABEL_FONT_SIZE)
    ax.set_xlabel("Date", fontsize=AX_LABEL_FONT_SIZE)

    ax.tick_params(axis='both', which='major', labelsize=TICK_LABEL_FONT_SIZE)

    # Save figure
    fig.tight_layout()
    fig.savefig(fname=f"{out_dir_path}/atmosphere_{variable.name}_{subset_time_start}_to_{subset_time_end}.png", format='png')